## Library Imports

Run this cell to import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import collections
import re
import gseapy as gp
from gseapy.plot import barplot, dotplot

import cptac
import cptac.algorithms as al

en = cptac.Endometrial()
ov = cptac.Ovarian()
co = cptac.Colon()

Checking that index is up-to-date...Loading acetylproteomics data...Loading clinical data...Loading CNA data...Loading definitions data...Loading miRNA data...Loading phosphoproteomics_gene data...Loading phosphoproteomics_site data...Loading proteomics data...Loading somatic_binary data...Loading somatic data...Loading transcriptomics_circular data...Loading transcriptomics_linear data...Formatting dataframes...Checking that index is up-to-date...Loading clinical data...Loading cnv data...Loading definitions data...Loading phosphoproteomics data...Loading proteomics data...Loading somatic_38 data...Loading transcriptomics data...Loading treatment data...Formatting dataframes...Checking that index is up-to-date...Loading clinical data...Loading miRNA data...Loading mutation_binary data...Loading mutation data...Loading phosphoproteomics_normal data...Loading phosphoproteomics_tumor data...Loading proteomics_normal data...Loading proteomics_tumor data...Loading transcriptomics data...Fo

## Get frequently mutated genes list

Enter the type of cancer and the cutoff for mutation frequency that you would like to use.

In [2]:
cancer_type = en
desired_cutoff = .15

fm = al.get_frequently_mutated(cancer_type, cutoff=desired_cutoff)
print(fm)

       Gene  Unique_Samples_Mut  Missence_Mut  Truncation_Mut
0     AHNAK            0.157895      0.126316        0.042105
1    ARID1A            0.452632      0.136842        0.400000
2   CCDC168            0.168421      0.115789        0.115789
3      CTCF            0.284211      0.094737        0.242105
4    CTNNB1            0.305263      0.305263        0.000000
5    DNAH17            0.157895      0.136842        0.031579
6     DOCK3            0.200000      0.115789        0.157895
7     FBXW7            0.189474      0.157895        0.042105
8     HERC2            0.178947      0.157895        0.063158
9     HUWE1            0.157895      0.157895        0.031579
10   INPPL1            0.200000      0.021053        0.189474
11     JAK1            0.168421      0.052632        0.126316
12    KMT2B            0.242105      0.115789        0.126316
13    KMT2D            0.178947      0.105263        0.084211
14     KRAS            0.326316      0.326316        0.000000
15    LM

## Select a gene from the list of frequently mutated genes above

Set the gene to which of the above frequently mutated genes you want to examine. For example, if you want to look at the gene PTEN, change the cell below to say "gene = 'PTEN'"

In [3]:
gene = 'MUC17'

## Select desired omics comparison

Select which type of omics you want to compare. If you want to compare proteomics data for the interacting proteins, uncomment the "omics = en.get_proteomics()" line, etc.

In [4]:
omics = en.get_proteomics()
#omics = en.get_transcriptomics()
#omics = en.get_phosphoproteomics()
#omics = en.get_acetylproteomics()

## Generate interacting proteins and test omics comparisons

Simply run this cell after following the instructions above to see the results of the comparisons

## Endometrial

In [5]:

'''Prevent slice degrees of freedom warning'''
import warnings
#warnings.filterwarnings("ignore")

print("\nGene: ", gene)

'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics.name == 'phosphoproteomics' or omics.name == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)
        
print(interacting_proteins_in_omics_df)


'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = en.append_mutations_to_omics(mutation_genes=[gene], omics_df_name="proteomics", omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the datafram correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")


Gene:  MUC17
Generating interacting protein list
['GALNT10', 'ST6GALNAC2', 'B3GNT3', 'ST6GALNAC3', 'GALNT12', 'ST3GAL2', 'GCNT3', 'MUC16', 'C1GALT1', 'MUC6', 'MUC5B', 'MUC1', 'MUC13', 'MUC5AC']
Doing t-test comparisons



/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


No significant comparisons.


## Ovarian

In [6]:

'''Prevent slice degrees of freedom warning'''
import warnings
#warnings.filterwarnings("ignore")

print("\nGene: ", gene)

'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics.name == 'phosphoproteomics' or omics.name == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)
        
print(interacting_proteins_in_omics_df)


'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = ov.append_mutations_to_omics(mutation_genes=[gene], omics_df_name="proteomics", omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the datafram correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")


Gene:  MUC17
Generating interacting protein list
['GALNT10', 'ST6GALNAC2', 'B3GNT3', 'ST6GALNAC3', 'GALNT12', 'ST3GAL2', 'GCNT3', 'MUC16', 'C1GALT1', 'MUC6', 'MUC5B', 'MUC1', 'MUC13', 'MUC5AC']
ST6GALNAC2 did not match any columns in proteomics dataframe. ST6GALNAC2_proteomics column inserted, but filled with NaN.
ST6GALNAC3 did not match any columns in proteomics dataframe. ST6GALNAC3_proteomics column inserted, but filled with NaN.
ST3GAL2 did not match any columns in proteomics dataframe. ST3GAL2_proteomics column inserted, but filled with NaN.
GCNT3 did not match any columns in proteomics dataframe. GCNT3_proteomics column inserted, but filled with NaN.
MUC6 did not match any columns in proteomics dataframe. MUC6_proteomics column inserted, but filled with NaN.
MUC13 did not match any columns in proteomics dataframe. MUC13_proteomics column inserted, but filled with NaN.
MUC5AC did not match any columns in proteomics dataframe. MUC5AC_proteomics column inserted, but filled with Na

AttributeError: 'BlockManager' object has no attribute 'T'

## Colon

In [7]:

'''Prevent slice degrees of freedom warning'''
import warnings
#warnings.filterwarnings("ignore")

print("\nGene: ", gene)

'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics.name == 'phosphoproteomics' or omics.name == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)
        
print(interacting_proteins_in_omics_df)


'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = co.append_mutations_to_omics(mutation_genes=[gene], omics_df_name="proteomics", omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the datafram correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")


Gene:  MUC17
Generating interacting protein list
['GALNT10', 'ST6GALNAC2', 'B3GNT3', 'ST6GALNAC3', 'GALNT12', 'ST3GAL2', 'GCNT3', 'MUC16', 'C1GALT1', 'MUC6', 'MUC5B', 'MUC1', 'MUC13', 'MUC5AC']
ST6GALNAC2 did not match any columns in proteomics dataframe. ST6GALNAC2_proteomics column inserted, but filled with NaN.
ST6GALNAC3 did not match any columns in proteomics dataframe. ST6GALNAC3_proteomics column inserted, but filled with NaN.
ST3GAL2 did not match any columns in proteomics dataframe. ST3GAL2_proteomics column inserted, but filled with NaN.
MUC16 did not match any columns in proteomics dataframe. MUC16_proteomics column inserted, but filled with NaN.
Doing t-test comparisons

No significant comparisons.


## Look at effect on all proteins

This will look at the effect of the selected gene mutation on all proteins, and report any significant results.

In [ ]:
'''
try:
    print("\nGene: ", gene)

    '''Use get interacting proteins method to generate list of interacting proteins'''
    proteomics = en.get_proteomics()
    proteins = proteomics.columns
    #proteins = proteins[:250]

    print("Generating protein list")
    interacting_proteins_in_omics_df = []

    '''Only do comparisons on proteins in the omics dataframe'''
    for ip in proteins:
        if omics.name == 'phosphoproteomics' or omics.name == 'acetylproteomics':
            col_regex = "^{}-.*$".format(ip) # Build a regex to get all columns that match the gene
        else:
            col_regex = '^{}$'.format(ip)

        selected = omics.filter(regex=col_regex)

        if len(selected.columns) > 0:
            interacting_proteins_in_omics_df.append(ip)

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = en.append_mutations_to_omics(mutation_genes=[gene], omics_df_name="proteomics", omics_genes=interacting_proteins_in_omics_df)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)
    
'''

## Gene Set Enrichment on Significant Omics Genes

In [ ]:
if len(wrap_results) > 0:
    '''Get a list of the omics genes deemed significant by wrap_ttest'''
    all_gene_list = list(wrap_results['Comparison'])
    gene_name_list = []
    for agl in all_gene_list:
        split = agl.split("_")
        gene_name_list.append(split[0])
    
    '''Use the gseapy library to run a gene set enrichment analysis on the resulting list of genes'''
    enrichment = gp.enrichr(gene_list = gene_name_list, description='ARID1A_Impacted', gene_sets='KEGG_2016', outdir='test/enrichr_kegg',cutoff=.5)
    print(enrichment.res2d)
    
else:
    print("No significant comparisons found.")

## Plot Gene Set Enrichment Results

In [ ]:
'''Plot the significant results of the gene set enrichment analysis'''
if len(wrap_results) > 0:
    barplot(enrichment.res2d, title=gene+ " Impacted Protein Enrichment Analysis")
else:
    print("No significant comparisons found.")